In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('.\data\FIT_news_embedded.csv')
df.info()

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\NguyenDuyDangKhoa\AppData\Local\Temp\ipykernel_3672\3950369667.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('.\data\FIT_news_embedded.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994 entries, 0 to 993
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        994 non-null    object
 1   article      994 non-null    object
 2   file_links   994 non-null    object
 3   url          994 non-null    object
 4   type         994 non-null    object
 5   created_at   994 non-null    object
 6   school_year  994 non-null    int64 
 7   in_effect    994 non-null    int64 
 8   chunks       994 non-null    object
 9   chunk_ids    994 non-null    int64 
 10  embedding    994 non-null    object
dtypes: int64(3), object(8)
memory usage: 85.6+ KB


In [9]:
pd.to_datetime(df['created_at'], format='%d/%m/%Y').dt.year.unique()

array([2024, 2023, 2020, 2022, 2021])

In [10]:
df['school_year'] = pd.to_datetime(df['created_at'], format='%d/%m/%Y').dt.year
df['is_active'] = True

In [13]:
df.head(2)

,title,article,file_links,url,type,created_at,school_year,in_effect,chunks,chunk_ids,embedding,is_active
0,Chương trình Hành trang tân sinh viên Khoa Côn...,\r\nI. LỊCH SINH HOẠT DÀNH CHO SINH VIÊN CHƯƠN...,[],https://www.fit.hcmus.edu.vn/tin-tuc/d/chuong-...,events,06/09/2024,2024,2024,I. LỊCH SINH HOẠT DÀNH CHO SINH VIÊN CHƯƠNG TR...,1,"[0.039748985, -0.011164816, -0.00078401115, -0...",True
1,Chương trình Hành trang tân sinh viên Khoa Côn...,\r\nI. LỊCH SINH HOẠT DÀNH CHO SINH VIÊN CHƯƠN...,[],https://www.fit.hcmus.edu.vn/tin-tuc/d/chuong-...,events,06/09/2024,2024,2024,16g30\r\nthứ bảy ngày 14/09/2024Địa điểm:\r\nG...,2,"[0.022049922, -0.0014875026, -0.0031239209, -0...",True


In [14]:
df['type'].unique()

array(['events', 'academic_affairs', 'scholarship', 'timetable',
       'recruitment'], dtype=object)

# Generate metadata

Events - location(1,2)  
academic affairs  
scholarship, recruitment - keywords(array of string), majors(array of string)  
timetable - subject(string)

In [15]:
WATSONX_APIKEY='ddhZhPOfHZvBDktnnZ10hSX6bUJRXfpNmtct-q3_Td0z'

WATSONX_PROJECT_ID='f1634c35-51e6-4d99-b617-010ac64db4d5'
model_id = "meta-llama/llama-3-1-70b-instruct"

from dotenv import load_dotenv
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

credentials = {
            "apikey": WATSONX_APIKEY,
            "url": "https://us-south.ml.cloud.ibm.com",
        }

model = ModelInference(
    model_id=model_id,
    credentials=credentials,
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MAX_NEW_TOKENS: 500,
        # GenParams.STOP_SEQUENCES: [],
    },
    project_id=WATSONX_PROJECT_ID,
)

## Generate location - events collection

In [76]:
prompt = """The following is a news article about an event at a school. Extract the location from the article if mentioned.
The school has two locations, with a few details about each location provided below:
Location 1: Cơ sở 1 - Cơ sở quận 5 - 227 Nguyễn Văn Cừ, Phường 4, Quận 5.
Location 2: Cơ sở 2 - Cơ sở Thủ Đức/Linh Trung - Làng Đại học, Phường Linh Trung, Thành phố Thủ Đức.
Return the number corresponding to the location. If no location of the two provided or both locations is mentioned, return 0. Note that the location may not be explicitly mentioned in full in the article.
Article: '''{article}'''.
Only answer with the number.
"""

In [90]:
sample = df[df['type'] == 'events'].sample()
sample['url'].values

array(['https://www.fit.hcmus.edu.vn/tin-tuc/d/dang-ky-tham-du-buoi-trao-doi-cac-phuong-thuc-thuc-hien-de-tai-danh-cho-hoc-vien-cao-hoc-khoa-32-2022-ngay-01-11-2023'],
      dtype=object)

In [91]:
full_prompt = prompt.format(article=sample['article'].values[0])
model.generate_text(full_prompt).replace('<|eom_id|>','')

'0'

In [93]:
full_prompts = [prompt.format(article=article) for article in df[df['type'] == 'events']['article'].values]

In [94]:
locations = model.generate_text(full_prompts)

In [95]:
len(locations)

266

In [98]:
locations = [int(float(l)) for l in locations]
locations

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 2,
 1,
 1,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 2,
 2,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 2,
 2,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 2,
 0,
 1,


In [99]:
df.loc[df['type'] == 'events', 'location'] = locations

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994 entries, 0 to 993
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        994 non-null    object 
 1   article      994 non-null    object 
 2   file_links   994 non-null    object 
 3   url          994 non-null    object 
 4   type         994 non-null    object 
 5   created_at   994 non-null    object 
 6   school_year  994 non-null    int32  
 7   in_effect    994 non-null    int64  
 8   chunks       994 non-null    object 
 9   chunk_ids    994 non-null    int64  
 10  embedding    994 non-null    object 
 11  is_active    994 non-null    bool   
 12  location     266 non-null    float64
dtypes: bool(1), float64(1), int32(1), int64(2), object(8)
memory usage: 90.4+ KB


## keywords, majors

In [135]:
prompt = """Extract the keywords mentioned in the following article.
Keywords are IT technological skills related to the article. Do not extract any other kinds of general keywords. Do not repeat keywords.
Article: '''{article}'''.
Output the keywords in a list, with a maximum of 5 keywords. Only answer with the list representation. If no keywords are mentioned, return an empty list.
"""

In [136]:
sample = df[df['type'].isin(['recruitment', 'scholarship'])].sample()
sample['url'].values[0]

'https://www.fit.hcmus.edu.vn/tin-tuc/d/lihuhu-unity_mobile-game-developer-vietnamese-ver-may2024'

In [137]:
full_prompt = prompt.format(article=sample['article'].values[0])
model.generate_text(full_prompt).replace('<|eom_id|>','')

"['Unity', 'Unity 3D Game Engine', 'Firebase', 'GIT', 'OOP']"

In [138]:
full_prompts = [prompt.format(article=article) for article in df[df['type'].isin(['recruitment', 'scholarship'])]['article'].values]

In [147]:
keywords = model.generate_text(full_prompts)
len(keywords)

308

In [148]:
keywords = [keyword.replace('<|eom_id|>','').lower() for keyword in keywords]
keywords

["['']",
 "['email']",
 "['công nghệ thông tin']",
 "['email']",
 "['email']",
 "['email']",
 "['cntt']",
 '[]',
 "['']",
 "['trí tuệ nhân tạo', 'khoa học', 'cntt', 'dữ liệu lớn', 'nghiên cứu']",
 "['trí tuệ nhân tạo', 'khoa học', 'cntt', 'dữ liệu lớn']",
 "['algorithmic trading', 'trading strategies', 'algorithmic trading systems']",
 "['html', 'css', 'oop', 'ruby', 'sql']",
 "['html', 'css', 'oop', 'ruby', 'sql']",
 "['frontend', 'backend', 'mobile', 'qa/qc', 'design']",
 "['frontend', 'backend', 'mobile', 'qa/qc', 'design']",
 "['it']",
 "['javascript', 'typescript', 'ruby', 'go', 'kotlin']",
 "['javascript', 'typescript', 'ruby', 'go', 'kotlin']",
 "['javascript', 'typescript', 'ruby', 'go', 'kotlin']",
 "['javascript', 'typescript', 'ruby', 'go', 'kotlin']",
 "['tcp/ip', 'routing', 'switching', 'mpls', 'osi']",
 "['dart', 'java', 'kotlin', 'c#', 'json']",
 "['cloud computing', 'rust', 'python']",
 "['kỹ thuật phần mềm', 'khoa học máy tính', 'khoa học thông tin', 'kỹ thuật truyền t

In [149]:
import ast 
keywords = [ast.literal_eval(keyword) for keyword in keywords]

In [153]:
keywords = [str(keyword) for keyword in keywords]

In [154]:
df.loc[df['type'].isin(['recruitment', 'scholarship']), 'keywords'] = keywords

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994 entries, 0 to 993
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        994 non-null    object 
 1   article      994 non-null    object 
 2   file_links   994 non-null    object 
 3   url          994 non-null    object 
 4   type         994 non-null    object 
 5   created_at   994 non-null    object 
 6   school_year  994 non-null    int32  
 7   in_effect    994 non-null    int64  
 8   chunks       994 non-null    object 
 9   chunk_ids    994 non-null    int64  
 10  embedding    994 non-null    object 
 11  is_active    994 non-null    bool   
 12  location     266 non-null    float64
 13  keywords     994 non-null    object 
dtypes: bool(1), float64(1), int32(1), int64(2), object(9)
memory usage: 98.2+ KB


In [156]:
df.to_csv('FIT_news_v2.csv', index=False)

## majors

In [163]:
majors_to_find = [
    'công nghệ phần mềm',
    'khoa học dữ liệu',
    'khoa học máy tính',
    'hệ thống thông tin',
    'kỹ thuật phần mềm',
    'công nghệ thông tin',
    'mạng máy tính',
    'thị giác máy tính',
    'công nghệ tri thức'
]

In [166]:
articles = df[df['type'].isin(['recruitment', 'scholarship'])]['article'].values
majors = [[word for word in majors_to_find if word in article.lower()] for article in articles]

In [168]:
len(majors)

308

In [170]:
majors = [str(major) for major in majors]

In [171]:
df.loc[df['type'].isin(['recruitment', 'scholarship']), 'majors'] = majors

Change "nan" strings to `np.nan`

In [183]:
import numpy as np

df['majors'] = df['majors'].replace('nan', np.nan)
df['keywords'] = df['keywords'].replace('nan', np.nan)

In [184]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994 entries, 0 to 993
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        994 non-null    object 
 1   article      994 non-null    object 
 2   file_links   994 non-null    object 
 3   url          994 non-null    object 
 4   type         994 non-null    object 
 5   created_at   994 non-null    object 
 6   school_year  994 non-null    int32  
 7   in_effect    994 non-null    int64  
 8   chunks       994 non-null    object 
 9   chunk_ids    994 non-null    int64  
 10  embedding    994 non-null    object 
 11  is_active    994 non-null    bool   
 12  location     266 non-null    float64
 13  keywords     308 non-null    object 
 14  majors       308 non-null    object 
dtypes: bool(1), float64(1), int32(1), int64(2), object(10)
memory usage: 105.9+ KB


## subject

subject information is sourced from: https://www.fit.hcmus.edu.vn/dai-hoc/thong-tin-mon-hoc

In [ ]:
df_subjects = pd.read_json('./data/thong-tin-mon-hoc.json')
df_subjects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    179 non-null    object 
 1   maMH                  179 non-null    object 
 2   code                  179 non-null    object 
 3   nameEN                179 non-null    object 
 4   nameVN                179 non-null    object 
 5   fullName              179 non-null    object 
 6   credits               179 non-null    int64  
 7   theory                179 non-null    int64  
 8   lab                   179 non-null    int64  
 9   ects                  179 non-null    int64  
 10  workload_Lec          179 non-null    int64  
 11  workload_Lab          179 non-null    int64  
 12  workload_SelfStudy    179 non-null    int64  
 13  semester              179 non-null    int64  
 14  lecturer              0 non-null      float64
 15  language              1

In [208]:
subjects_name_ref = df_subjects['nameVN'].str.lower().values.tolist() + df_subjects['nameEN'].str.lower().values.tolist()
subjects_code_ref = df_subjects['maMH'].str.lower().values.tolist()

In [211]:
articles = df[df['type'] == 'timetable']['article'].values
subjects_name = [[word for word in subjects_name_ref if word in article.lower()] for article in articles]
subjects_code = [[word for word in subjects_code_ref if word in article.lower()] for article in articles]

In [213]:
len(subjects_name), len(subjects_code)

(107, 107)

In [216]:
subjects_name = [str(subject) for subject in subjects_name]
subjects_code = [str(subject) for subject in subjects_code]

In [217]:
df.loc[df['type'] == 'timetable', 'subjects_name'] = subjects_name
df.loc[df['type'] == 'timetable', 'subjects_code'] = subjects_code

In [219]:
import numpy as np

df['subjects_name'] = df['subjects_name'].replace('nan', np.nan)
df['subjects_code'] = df['subjects_code'].replace('nan', np.nan)

In [223]:
df.to_csv('FIT_news_v2.csv', index=False)

# Add chunk number

In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994 entries, 0 to 993
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          994 non-null    object 
 1   article        994 non-null    object 
 2   file_links     994 non-null    object 
 3   url            994 non-null    object 
 4   type           994 non-null    object 
 5   created_at     994 non-null    object 
 6   school_year    994 non-null    int32  
 7   in_effect      994 non-null    int64  
 8   chunks         994 non-null    object 
 9   chunk_ids      994 non-null    int64  
 10  embedding      994 non-null    object 
 11  is_active      994 non-null    bool   
 12  location       266 non-null    float64
 13  keywords       308 non-null    object 
 14  majors         308 non-null    object 
 15  subjects_name  107 non-null    object 
 16  subjects_code  107 non-null    object 
dtypes: bool(1), float64(1), int32(1), int64(2), object(12)

In [227]:
df.columns = ['title', 'article', 'file_links', 'url', 'type', 'created_at',
       'school_year', 'in_effect', 'chunks', 'chunk_id', 'embedding',
       'is_active', 'location', 'keywords', 'majors', 'subjects_name',
       'subjects_code']

In [228]:
df.to_csv('data/FIT_news_v2.csv', index=False)